In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Load flight data
df = pd.read_excel("flight_prices_16072025.xlsx")


# Clean and convert
df = df.dropna(subset=["departure", "arrival_airport", "departure_airport", "price"])
df["departure"] = pd.to_datetime(df["departure"])
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df = df.dropna(subset=["DepartureCity", "ArrivalCity"])

# ---- USER INPUT ----

# Outbound: from Polish cities to these arrival cities
allowed_arrival_cities = ["Valencia", "Madrid", "Malaga", "Alicante", "Seville", "Palma de Mallorca", "Faro", "Lisbon", "Porto"]  # User-defined arrival cities

# Inbound: from these departure cities to Polish cities
allowed_departure_cities = ["Valencia", "Madrid", "Malaga", "Alicante", "Seville", "Palma de Mallorca", "Faro", "Lisbon", "Porto"]   # User-defined departure cities

# Define Polish cities
polish_cities = ["Warsaw", "Krakow", "Gdansk", "Katowice", "Poznan", "Wroclaw"]

# Trip duration limits
min_days = 5
max_days = 8

# ----------------------

# OUTBOUND: from Polish city to selected arrival cities
outbound_df = df[
    (df["DepartureCity"].isin(polish_cities)) &
    (df["ArrivalCity"].isin(allowed_arrival_cities))
].copy()

# INBOUND: from selected departure cities to Polish cities
inbound_df = df[
    (df["DepartureCity"].isin(allowed_departure_cities)) &
    (df["ArrivalCity"].isin(polish_cities))
].copy()

# Combine outbound and inbound flights
results = []

for i, out_row in outbound_df.iterrows():
    for j, in_row in inbound_df.iterrows():
        # Trip duration check
        days_diff = (in_row["departure"] - out_row["departure"]).days
        if days_diff < min_days or days_diff > max_days:
            continue

        # City-to-city must differ
        if out_row["ArrivalCity"] == in_row["DepartureCity"]:
            continue

        results.append({
            "Outbound From": out_row["DepartureCity"],
            "Outbound To": out_row["ArrivalCity"],
            "Outbound Date": out_row["departure"].date(),
            "Inbound From": in_row["DepartureCity"],
            "Inbound To": in_row["ArrivalCity"],
            "Inbound Date": in_row["departure"].date(),
            "Trip Duration (Days)": days_diff,
            "Outbound Price": out_row["price"],
            "Inbound Price": in_row["price"],
            "Total Price": out_row["price"] + in_row["price"]
        })

# Save to Excel
results_df = pd.DataFrame(results)
results_df = results_df.sort_values("Total Price").reset_index(drop=True)
results_df.to_excel("multi_city_trips_by_city.xlsx", index=False)

print("Results saved to 'multi_city_trips_by_city.xlsx'")


Results saved to 'multi_city_trips_by_city.xlsx'


In [7]:
#TKINTER_APP
import pandas as pd
import tkinter as tk
from tkinter import ttk
from datetime import datetime

# === Hardcoded Polish Cities ===
polish_cities = ["Warsaw", "Krakow", "Gdansk", "Katowice", "Poznan", "Wroclaw"]

# === Load airport_data from your static definition ===
airport_data = [
    {"IATA": "HER", "City": "Heraklion", "Country": "Greece"},
    {"IATA": "RHO", "City": "Rhodes", "Country": "Greece"},
    {"IATA": "SKG", "City": "Thessaloniki", "Country": "Greece"},
    {"IATA": "CFU", "City": "Corfu", "Country": "Greece"},
    {"IATA": "ZTH", "City": "Zakynthos", "Country": "Greece"},
    {"IATA": "KGS", "City": "Kos", "Country": "Greece"},
    {"IATA": "CHQ", "City": "Chania", "Country": "Greece"},
    {"IATA": "MAD", "City": "Madrid", "Country": "Spain"},
    {"IATA": "BCN", "City": "Barcelona", "Country": "Spain"},
    {"IATA": "AGP", "City": "Malaga", "Country": "Spain"},
    {"IATA": "PMI", "City": "Palma de Mallorca", "Country": "Spain"},
    {"IATA": "ALC", "City": "Alicante", "Country": "Spain"},
    {"IATA": "TFS", "City": "Tenerife", "Country": "Spain"},
    {"IATA": "SVQ", "City": "Seville", "Country": "Spain"},
    {"IATA": "IBZ", "City": "Ibiza", "Country": "Spain"},
    {"IATA": "VLC", "City": "Valencia", "Country": "Spain"},
    {"IATA": "BIO", "City": "Bilbao", "Country": "Spain"},
    {"IATA": "TFN", "City": "Tenerife", "Country": "Spain"},
    {"IATA": "SPC", "City": "La Palma", "Country": "Spain"},
    {"IATA": "GRX", "City": "Granada", "Country": "Spain"},
    {"IATA": "FCO", "City": "Rome", "Country": "Italy"},
    {"IATA": "CIA", "City": "Rome", "Country": "Italy"},
    {"IATA": "MXP", "City": "Milan", "Country": "Italy"},
    {"IATA": "BGY", "City": "Milan", "Country": "Italy"},
    {"IATA": "VCE", "City": "Venice", "Country": "Italy"},
    {"IATA": "NAP", "City": "Naples", "Country": "Italy"},
    {"IATA": "BLQ", "City": "Bologna", "Country": "Italy"},
    {"IATA": "PSA", "City": "Pisa", "Country": "Italy"},
    {"IATA": "FLR", "City": "Florence", "Country": "Italy"},
    {"IATA": "TRN", "City": "Turin", "Country": "Italy"},
    {"IATA": "BRI", "City": "Bari", "Country": "Italy"},
    {"IATA": "PMO", "City": "Palermo", "Country": "Italy"},
    {"IATA": "CTA", "City": "Catania", "Country": "Italy"},
    {"IATA": "OLB", "City": "Olbia", "Country": "Italy"},
    {"IATA": "AHO", "City": "Alghero", "Country": "Italy"},
    {"IATA": "VRN", "City": "Verona", "Country": "Italy"},
    {"IATA": "GOA", "City": "Genoa", "Country": "Italy"},
    {"IATA": "SPU", "City": "Split", "Country": "Croatia"},
    {"IATA": "DBV", "City": "Dubrovnik", "Country": "Croatia"},
    {"IATA": "ZAD", "City": "Zadar", "Country": "Croatia"},
    {"IATA": "LCA", "City": "Larnaca", "Country": "Cyprus"},
    {"IATA": "PFO", "City": "Paphos", "Country": "Cyprus"},
    {"IATA": "VAR", "City": "Varna", "Country": "Bulgaria"},
    {"IATA": "BOJ", "City": "Burgas", "Country": "Bulgaria"},
    {"IATA": "TGD", "City": "Podgorica", "Country": "Montenegro"},
    {"IATA": "TIA", "City": "Tirana", "Country": "Albania"},
    {"IATA": "FAO", "City": "Faro", "Country": "Portugal"},
    {"IATA": "LIS", "City": "Lisbon", "Country": "Portugal"},
    {"IATA": "OPO", "City": "Porto", "Country": "Portugal"},
    {"IATA": "FNC", "City": "Madeira", "Country": "Portugal"},
    {"IATA": "CDG", "City": "Paris", "Country": "France"},
    {"IATA": "ORY", "City": "Paris", "Country": "France"},
    {"IATA": "BVA", "City": "Paris", "Country": "France"},
    {"IATA": "LYS", "City": "Lyon", "Country": "France"},
    {"IATA": "NCE", "City": "Nice", "Country": "France"},
    {"IATA": "MRS", "City": "Marseille", "Country": "France"},
    {"IATA": "BIQ", "City": "Biarritz", "Country": "France"},
    {"IATA": "KEF", "City": "Reykjavik", "Country": "Iceland"},
    {"IATA": "RMO", "City": "Chisinau", "Country": "Moldova"},
    {"IATA": "BBU", "City": "Bucharest", "Country": "Romania"},
    {"IATA": "OTP", "City": "Bucharest", "Country": "Romania"},
    {"IATA": "CPH", "City": "Copenhagen", "Country": "Denmark"},
    {"IATA": "ARN", "City": "Stockholm", "Country": "Sweden"},
    {"IATA": "GOT", "City": "Gothenburg", "Country": "Sweden"},
    {"IATA": "ATH", "City": "Athens", "Country": "Greece"},
    {"IATA": "BUD", "City": "Budapest", "Country": "Hungary"},
    {"IATA": "AUH", "City": "Abu Dhabi", "Country": "United Arab Emirates"},
    {"IATA": "IST", "City": "Istanbul", "Country": "Turkey"},
    {"IATA": "SKP", "City": "Skopje", "Country": "North Macedonia"},
    {"IATA": "KUT", "City": "Kutaisi", "Country": "Georgia"},
    {"IATA": "MLA", "City": "Valletta", "Country": "Malta"},
    {"IATA": "AMM", "City": "Amman", "Country": "Jordan"},
    {"IATA": "RAK", "City": "Marrakesh", "Country": "Morocco"},
    {"IATA": "AGA", "City": "Agadir", "Country": "Morocco"},
    {"IATA": "RBA", "City": "Rabat", "Country": "Morocco"},
]

# Extract unique cities
other_cities = sorted(set(entry["City"] for entry in airport_data))

# === Load Excel once ===
df = pd.read_excel("flight_prices_16072025.xlsx")
df = df.dropna(subset=["departure", "arrival_airport", "departure_airport", "price", "DepartureCity", "ArrivalCity"])
df["departure"] = pd.to_datetime(df["departure"])
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df = df.dropna(subset=["price"])

# === Processing Function ===
def process_flights():
    selected_polish = [polish_listbox.get(i) for i in polish_listbox.curselection()]
    selected_arrivals = [arrival_listbox.get(i) for i in arrival_listbox.curselection()]
    selected_departures = [departure_listbox.get(i) for i in departure_listbox.curselection()]

    try:
        min_days = int(min_days_var.get())
        max_days = int(max_days_var.get())
    except ValueError:
        result_label.config(text="Min/Max days must be valid integers", foreground="red")
        return

    outbound_df = df[
        (df["DepartureCity"].isin(selected_polish)) &
        (df["ArrivalCity"].isin(selected_arrivals))
    ].copy()

    inbound_df = df[
        (df["DepartureCity"].isin(selected_departures)) &
        (df["ArrivalCity"].isin(selected_polish))
    ].copy()

    results = []
    for _, out_row in outbound_df.iterrows():
        for _, in_row in inbound_df.iterrows():
            days_diff = (in_row["departure"] - out_row["departure"]).days
            if days_diff < min_days or days_diff > max_days:
                continue
            if out_row["ArrivalCity"] == in_row["DepartureCity"]:
                continue

            results.append({
                "Outbound From": out_row["DepartureCity"],
                "Outbound To": out_row["ArrivalCity"],
                "Outbound Date": out_row["departure"].date(),
                "Inbound From": in_row["DepartureCity"],
                "Inbound To": in_row["ArrivalCity"],
                "Inbound Date": in_row["departure"].date(),
                "Trip Duration (Days)": days_diff,
                "Outbound Price": out_row["price"],
                "Inbound Price": in_row["price"],
                "Total Price": out_row["price"] + in_row["price"]
            })

    if not results:
        result_label.config(text="No matching trips found.", foreground="orange")
        return

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values("Total Price").reset_index(drop=True)
    results_df.to_excel("multi_city_trips_by_city.xlsx", index=False)
    result_label.config(text="Results saved to 'multi_city_trips_by_city.xlsx'", foreground="green")

# === GUI Setup ===
root = tk.Tk()
root.title("Flight Search GUI")

main_frame = ttk.Frame(root, padding=10)
main_frame.pack(fill="both", expand=True)

# Listbox helper
def create_listbox(label_text, items, row, column):
    ttk.Label(main_frame, text=label_text).grid(row=row, column=column, sticky="w")
    box = tk.Listbox(main_frame, selectmode=tk.MULTIPLE, height=10, exportselection=False)
    for city in items:
        box.insert(tk.END, city)
    box.grid(row=row+1, column=column)
    return box

polish_listbox = create_listbox("Polish Cities:", polish_cities, 0, 0)
arrival_listbox = create_listbox("Arrival Cities (Outbound):", other_cities, 0, 1)
departure_listbox = create_listbox("Departure Cities (Inbound):", other_cities, 0, 2)

# Trip duration
ttk.Label(main_frame, text="Trip Duration (Days):").grid(row=2, column=0, columnspan=3, pady=(10, 0), sticky="w")
min_days_var = tk.StringVar(value="5")
max_days_var = tk.StringVar(value="8")
ttk.Label(main_frame, text="Min:").grid(row=3, column=0, sticky="w")
ttk.Entry(main_frame, textvariable=min_days_var, width=5).grid(row=3, column=0, sticky="e")
ttk.Label(main_frame, text="Max:").grid(row=3, column=1, sticky="w")
ttk.Entry(main_frame, textvariable=max_days_var, width=5).grid(row=3, column=1, sticky="e")

# Button
ttk.Button(main_frame, text="Find Trips", command=process_flights).grid(row=4, column=0, columnspan=3, pady=10)

# Result Label
result_label = ttk.Label(main_frame, text="", wraplength=600)
result_label.grid(row=5, column=0, columnspan=3, sticky="w")

root.mainloop()
